<a href="https://colab.research.google.com/github/yishairasowsky/Sefaria-Project/blob/master/ML/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd
import shutil

In [0]:
encoding = "latin-1"
# encoding = 'unicode_escape'
# encoding = "ISO-8859-1"
# encoding = "ISO-8859-8"
# encoding = "cp1255"

full_df = pd.read_csv(
    '/content/drive/My Drive/Colab Datasets/topic_data.csv',
    encoding=encoding,
    error_bad_lines=False
    )[:5000]

In [33]:
print("Original shape:",full_df.shape)

full_df.dropna(inplace=True)
print("Without nulls",full_df.shape)

full_df.drop_duplicates(inplace=True)
print("Without duplicates",full_df.shape)
full_df.head(5)


Original shape: (3294, 5)
Without nulls (3294, 5)
Without duplicates (3294, 5)


,Ref,En,He,Topics,Expanded Topics
0,Bekhorot 28a:5,Â§ The mishna teaches: If <b>a blemish develop...,× ××× ×× ××× ××ª×× ×©× ×ª× ×¨×©××...,laws-of-animal-firstlings,laws-of-animal-firstlings laws-of-worship-of-g...
1,Sforno on Genesis 21:22:1,"××××§×× ×¢××, seeing that Gâd clearly...",<b>××××× ×¢××</b> ××¤××× ×× × ××...,abraham-and-abimelech,entity specific-biblical-person-relationship r...
3,Gittin 78a:1,"<b>a modifier,</b> meaning that this board is ...",×©× ×××× ××× ×××ª ××× ×©× ×××...,dinei-haget,entity halakhah continuant laws mitzvot generi...
5,Sanhedrin 2a:5,The <b>intercalation of the month</b> is perfo...,×¢××××¨ ××××© ××©××©× ×¢××××¨ ××©...,laws-of-judges-and-courts judgements1 laws-of-...,beit-din group-of-people independent-continuan...
6,Pesachim 78a:15,<b>The statement of Rabbi Eliezer</b> is corre...,×××¨× ×¨×× ××××¢××¨ ×××××× ×©×...,torah-study-and-a-livelihood life,lifecycle-events entity philosophy religion to...


In [40]:
import pkl
from bs4 import BeautifulSoup
import re
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords

STOPWORDS = set(stopwords.words('english'))

def preprocess_text(sentence):

    # HTML decoding
    sentence = BeautifulSoup(sentence, "lxml").text 
    
    # lowercase text
    sentence = sentence.lower() 

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    # Removing stopwords
    sentence = ' '.join(word for word in sentence.split() if word not in STOPWORDS) # delete stopwors from text

    return sentence

clean_df = full_df[['En','Topics']]
clean_df.En = clean_df.En.apply(preprocess_text)
clean_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,En,Topics
0,mishna teaches blemish developed within first ...,laws-of-animal-firstlings
1,seeing clearly side afraid afraid personal str...,abraham-and-abimelech
3,modifier meaning board referred unique name mo...,dinei-haget
5,intercalation month performed panel three judg...,laws-of-judges-and-courts judgements1 laws-of-...
6,statement rabbi eliezer correct regard animal ...,torah-study-and-a-livelihood life


In [0]:
clean_df.to_pickle("/content/drive/My Drive/Colab Datasets/clean_df.pkl")

In [42]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

stem_df = clean_df
stem_df.En = stem_df.En.apply(stemming)
stem_df.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,En,Topics
0,mishna teach blemish develop within first year...,laws-of-animal-firstlings
1,see clear side afraid afraid person strength a...,abraham-and-abimelech
3,modifi mean board refer uniqu name modifi even...,dinei-haget
5,intercal month perform panel three judg interc...,laws-of-judges-and-courts judgements1 laws-of-...
6,statement rabbi eliez correct regard anim offe...,torah-study-and-a-livelihood life


In [0]:
stem_df.to_pickle("/content/drive/My Drive/Colab Datasets/stem_df.pkl")

In [46]:
eng_df = stem_df[['En','Topics']]
topics_df = eng_df.join(eng_df.pop('Topics').str.get_dummies(' '))
topics_df.sample()

,En,aaron,abandoned-corpses,abaye,abba-binyamin,abel,abels-sacrifice,abiathar,abiel,abihu1,abimelech-and-sarah,abodes,abraham,abraham-and-abimelech,abraham-and-his-environment,abraham-and-lot,abraham-and-sarah,abraham-and-sodom,abraham-and-the-angels,abraham-and-the-kings,abraham-in-egypt,abraham-in-ur-kasdim-and-his-departure-from-there,absalom,acceptance-of-nazarite-vows-and-its-duration,acquisitions,acronyms,acts-of-god,actuality,adam,adams-sin-and-his-punishment,additional-gifts-to-the-priesthood,adonijah,adornments,afflictions,agency,agriculture-(×¢××××ª-××××),ahab,ahasuerus,ahasuerus-and-esther,ahasuerus-party,...,workers,worlds,worms,wounds-(×¤×¦×¢),wrapping,writing,writing-a-torah-scroll,yalta,yannai,yavesh-gilad,yavneh,years,yehonatan-ben-gershom,yehoram-(king-of-israel),yehudah-b-tabbai,yell,yellow,yerovam-(son-of-jehoash),yerovam-(son-of-nebat),yetzer-hara,yetzer-hatov,yidoni,yirat-shamayim,yitro,yoel,yom-kippur,yom-kippur-eve,yom-kippur-offerings,yom-tov,yom-tov-sheini,young-men,young-torah-scholars,zaken-mamre,zavah,zeal,zechariah,zedekiah-the-son-of-chenaanah,zimri,zion,zionism
2533,gemara regard one play dice money disqualifi b...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
topics_df.to_pickle("/content/drive/My Drive/Colab Datasets/topics_df.pkl")